In [1]:
import numpy as np 
import pandas as pd 
import string 
import torch 
import torch.nn as nn 
from transformers import BertPreTrainedModel, BertModel, BertTokenizerFast, AdamW
from colbert.parameters import DEVICE 
import os 
import pickle

# Create Indexes for corpus

In [3]:
!python -m colbert.index --amp --doc_maxlen 512 --query_maxlen 512 \
--mask-punctuation --bsize 256 --checkpoint ./experiments/dirty/train.py/2021-12-07_09.33.51/checkpoints/colbert-40000.dnn \
--collection test_collections.tsv --index_root ./experiments/indexes --index_name large_train_index

2021-12-08 08:33:30.224419: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


[Dec 08, 08:33:31] #> Creating directory /tf/document_similarity/ColBERT/experiments/dirty/index.py/2021-12-08_08.33.29 


INFO: 'dirty/index.py' does not exist. Creating a new experiment


[Dec 08, 08:33:32] #> Creating directory /tf/document_similarity/ColBERT/experiments/dirty/index.py/2021-12-08_08.33.29/logs/ 


[Dec 08, 08:33:32] {'root': 'experiments', 'experiment': 'dirty', 'run': '2021-12-08_08.33.29', 'rank': -1, 'similarity': 'cosine', 'dim': 128, 'query_maxlen': 512, 'doc_maxlen': 512, 'mask_punctuation': True, 'checkpoint': './experiments/dirty/train.py/2021-12-07_09.33.51/checkpoints/colbert-40000.dnn', 'bsize': 256, 'amp': True, 'collection': 'test_collections.tsv', 'index_root': './experiments/indexes', 'index_name': 'large_train_index', 'chunksize': 6.0} 



[Dec 08, 08:33:32] #> Creating directory ./experiments/indexes 




# FAISS Indexing for End-To-End Retrieval

In [4]:
!python -m colbert.index_faiss --index_root ./experiments/indexes --index_name large_train_index 



[Dec 08, 08:48:51] #> Creating directory /tf/document_similarity/ColBERT/experiments/dirty/index_faiss.py/2021-12-08_08.48.51 


INFO: 'dirty/index_faiss.py' does not exist. Creating a new experiment


[Dec 08, 08:48:51] #> Creating directory /tf/document_similarity/ColBERT/experiments/dirty/index_faiss.py/2021-12-08_08.48.51/logs/ 


[Dec 08, 08:48:51] {'root': 'experiments', 'experiment': 'dirty', 'run': '2021-12-08_08.48.51', 'rank': -1, 'index_root': './experiments/indexes', 'index_name': 'large_train_index', 'partitions': None, 'sample': None, 'slices': 1} 

#> num_embeddings = 38915192



[Dec 08, 08:48:51] [WARNING] 	 You did not specify --partitions!
[Dec 08, 08:48:51] [WARNING] 	 Default computation chooses 65536 partitions (for 38915192 embeddings)



[Dec 08, 08:48:51] #> Starting..
[Dec 08, 08:48:51] #> Processing slice #1 of 1 (range 0..2).
[Dec 08, 08:48:51] #> Will write to ./experiments/indexes/large_train_index/ivfpq.65536.faiss.
[Dec 08, 08:48:51] #> Loading ./exper

# Retrieve and ReRank

Attempt for the first 100 queries 

In [18]:
test_queries = pd.read_csv("test_queries.tsv",sep="\t") 

test_queries.head(3)

,id,queries
0,1,A system for charging or maintaining a batter...
1,2,A method for calculating a path delay in stat...
2,3,A preventive or therapeutic agent for diabete...


In [19]:
small_test_queries = test_queries.iloc[:100,:] 

small_test_queries.to_csv("small_test_queries.tsv", index=False, sep="\t") 

In [20]:
test_collections = pd.read_csv("test_collections.tsv", sep="\t")  

test_collections.head(3) 

,id,passages
0,1,A method for battery charger and diagnosis wi...
1,2,A non transitory computer readable medium car...
2,3,A method for measuring glutamyl transpeptidas...


In [21]:
small_test_queries.shape, test_collections.shape

((100, 2), (77886, 2))

In [11]:
!python -m colbert.retrieve --amp --doc_maxlen 512 --query_maxlen 512 --bsize 256 \
--queries small_test_queries.tsv --partitions 65536\
--index_root ./experiments/indexes --index_name large_train_index \
--collection test_collections.tsv \
--checkpoint ./experiments/dirty/train.py/2021-12-07_09.33.51/checkpoints/colbert-40000.dnn --batch --retrieve_only 

2021-12-08 09:31:24.717369: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


[Dec 08, 09:31:26] #> Creating directory /tf/document_similarity/ColBERT/experiments/dirty/retrieve.py/2021-12-08_09.31.24 




[Dec 08, 09:31:26] #> Creating directory /tf/document_similarity/ColBERT/experiments/dirty/retrieve.py/2021-12-08_09.31.24/logs/ 


[Dec 08, 09:31:26] {'root': 'experiments', 'experiment': 'dirty', 'run': '2021-12-08_09.31.24', 'rank': -1, 'similarity': 'cosine', 'dim': 128, 'query_maxlen': 512, 'doc_maxlen': 512, 'mask_punctuation': False, 'checkpoint': './experiments/dirty/train.py/2021-12-07_09.33.51/checkpoints/colbert-40000.dnn', 'bsize': 256, 'amp': True, 'queries': 'small_test_queries.tsv', 'collection': 'test_collections.tsv', 'qrels': None, 'index_root': './experiments/indexes', 'index_name': 'large_train_index', 'partitions': 65536, 'nprobe': 10, 'retrieve_only': True, 'faiss_name': None, 'faiss_depth': 1

In [6]:
!python -m colbert.rerank --batch --log-scores --topk ./experiments/dirty/retrieve.py/2021-12-08_09.31.24/unordered.tsv \
--query_maxlen 512 --doc_maxlen 512 --mask-punctuation \
--checkpoint ./experiments/dirty/train.py/2021-12-07_09.33.51/checkpoints/colbert-40000.dnn \
--amp --queries small_test_queries.tsv \
--index_root ./experiments/indexes --index_name large_train_index --bsize 8

2021-12-08 09:46:03.224803: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


[Dec 08, 09:46:04] #> Creating directory /tf/document_similarity/ColBERT/experiments/dirty/rerank.py/2021-12-08_09.46.03 




[Dec 08, 09:46:04] #> Creating directory /tf/document_similarity/ColBERT/experiments/dirty/rerank.py/2021-12-08_09.46.03/logs/ 


[Dec 08, 09:46:04] {'root': 'experiments', 'experiment': 'dirty', 'run': '2021-12-08_09.46.03', 'rank': -1, 'similarity': 'cosine', 'dim': 128, 'query_maxlen': 512, 'doc_maxlen': 512, 'mask_punctuation': True, 'checkpoint': './experiments/dirty/train.py/2021-12-07_09.33.51/checkpoints/colbert-40000.dnn', 'bsize': 8, 'amp': True, 'queries': 'small_test_queries.tsv', 'collection': None, 'qrels': None, 'topK': './experiments/dirty/retrieve.py/2021-12-08_09.31.24/unordered.tsv', 'shortcircuit': False, 'index_root': './experiments/indexes', 'index_name': 'large_train_index', 'partitions': None,

[Dec 08, 09:49:52] #> Processing batch #2200..
[Dec 08, 09:49:54] #> Processing batch #2300..
[Dec 08, 09:49:57] #> Processing batch #2400..
[Dec 08, 09:49:59] #> Processing batch #2500..
[Dec 08, 09:50:02] #> Processing batch #2600..
[Dec 08, 09:50:04] #> Processing batch #2700..
[Dec 08, 09:50:06] #> Logging ranked lists to /tf/document_similarity/ColBERT/experiments/dirty/rerank.py/2021-12-08_09.46.03/ranking.tsv
[Dec 08, 09:50:06] #> Logging query #0 (qid 1) now...
#> ( QID 1 ) 1)  10795 : 486.26324462890625      None
#> ( QID 1 ) 2)  2853 : 485.85107421875      None



/tf/document_similarity/ColBERT/experiments/dirty/rerank.py/2021-12-08_09.46.03/ranking.tsv
[Dec 08, 09:50:07] #> Done.



# Evaluation: MRR (Mean Reciprocal Rank) for 100 queries

In [7]:
rankings = pd.read_csv("experiments/dirty/rerank.py/2021-12-08_09.46.03/ranking.tsv", sep="\t") 

rankings.head()

,1,10795,1.1,486.26324462890625
0,1,2853,2,485.851074
1,1,21272,3,485.454071
2,1,31073,4,484.933655
3,1,28206,5,484.728210
4,1,31792,6,484.728210


In [8]:
rankings = rankings.columns.to_frame().T.append(rankings, ignore_index=True) 

rankings.columns = range(len(rankings.columns)) 

rankings.head()

,0,1,2,3
0,1,10795,1.1,486.26324462890625
1,1,2853,2,485.851
2,1,21272,3,485.454
3,1,31073,4,484.934
4,1,28206,5,484.728


In [35]:
query_texts = small_test_queries['queries'].values 

In [38]:
### load dictionary that contains (query text, similar document list) 

with open('query_positive_dict.pkl', 'rb') as f: 
    loaded_dict = pickle.load(f)  

In [43]:
def calc_ranks(df, idx):
    relevant = loaded_dict[query_texts[idx-1]] 
    rank_i = df[df[0]==idx] 
    for i, val in enumerate(rank_i[1].values): 
        if val in relevant: 
            return i 

In [44]:
### MRR calculation 
ranks = [] 
for i in range(1,small_test_queries.shape[0]+1):  
    ranks.append(calc_ranks(rankings, i))  

s = 0 
for r in ranks: 
    if r is None: 
        s += 0 
    else: 
        s += 1/(r+1)  

s = s / len(ranks)

print("MRR for {} queries = {:.3f}".format(len(ranks), s))

MRR for 100 queries = 0.212
